# Accessing IPPS Case Rate Data on data.cms.gov Using Socrata API

All data is accessed from this page via the Socrata endpoint https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/fm2n-hjj6/data

Pulling & querying the API endpoint is based on these open source examples from Socrata
* basic queries: https://github.com/xmunoz/sodapy/blob/master/examples/basic_queries.ipynb
* filter queries: https://github.com/xmunoz/sodapy/blob/master/examples/soql_queries.ipynb

In [41]:
# import libraries
import pandas as pd
import os
import numpy as np
from sodapy import Socrata

In [42]:
ipps_soda_endpoint = "https://data.cms.gov/resource/t8zw-d33c.json"
ipps_soda_endpoint_domain = "data.cms.gov/"
ipps_dataset_identifier = "t8zw-d33c"

In [43]:
socrata_token = os.environ.get("SODAPY_APPTOKEN")

In [44]:
client = Socrata(ipps_soda_endpoint_domain, socrata_token)

In [45]:
results = client.get(ipps_dataset_identifier)

In [46]:
# sample the first json record
results[0]

{'drg_definition': '001 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W MCC',
 'provider_id': '100007',
 'provider_name': 'FLORIDA HOSPITAL',
 'provider_street_address': '601 E ROLLINS ST',
 'provider_city': 'ORLANDO',
 'provider_state': 'FL',
 'provider_zip_code': '32803',
 'hospital_referral_region_hrr_description': 'FL - Orlando',
 'total_discharges': '36',
 'average_covered_charges': '1126268.58',
 'average_total_payments': '189464.00',
 'average_medicare_payments': '182770.86'}

In [47]:
# put the results from API into pandas dataframe
df = pd.DataFrame.from_dict(results)
df.head()

,average_covered_charges,average_medicare_payments,average_total_payments,drg_definition,hospital_referral_region_hrr_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,1126268.58,182770.86,189464.00,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,FL - Orlando,ORLANDO,100007,FLORIDA HOSPITAL,FL,601 E ROLLINS ST,32803,36
1,1050063.13,252782.53,301640.47,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,FL - Miami,MIAMI,100022,JACKSON MEMORIAL HOSPITAL,FL,1611 NW 12TH AVE,33136,15
2,569379.33,199879.20,211545.13,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,FL - Gainesville,GAINESVILLE,100113,UF HEALTH SHANDS HOSPITAL,FL,1600 SW ARCHER RD,32610,15
3,1058968.75,189331.79,198471.08,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,FL - Tampa,TAMPA,100128,TAMPA GENERAL HOSPITAL,FL,1 TAMPA GENERAL CIR,33606,24
4,802446.90,188267.90,321528.10,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,FL - Jacksonville,JACKSONVILLE,100151,MAYO CLINIC,FL,4500 SAN PABLO RD,32224,20


In [48]:
# results are limited to 1000 without an API token from CMS. 
df.shape

(1000, 12)

### Call the API & Filter for NC records only

* In this section, we can use SQL like query parameters and pass into the client.get method to filter the API call & results

In [49]:
# params 
provider_state = input("enter a state code: ")
provider_id = input("enter a provider_id (MedParID): ")

enter a state code: NC
enter a provider_id (MedParID): 340090


In [50]:
sql_query = """
select *
where provider_state ="""+"""'"""+provider_state+"""'"""+"""
and provider_id ="""+"""'"""+provider_id+"""'"""+"""
"""
# passe the SQL query
nc_results = client.get(ipps_dataset_identifier,query=sql_query)

In [51]:
print(sql_query)


select *
where provider_state ='NC'
and provider_id ='340090'



In [52]:
nc_df = pd.DataFrame.from_dict(nc_results)
nc_df.head()

,average_covered_charges,average_medicare_payments,average_total_payments,drg_definition,hospital_referral_region_hrr_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,27282.83,5781.33,7162.83,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,NC - Raleigh,SMITHFIELD,340090,JOHNSTON HEALTH,NC,509 BRIGHT LEAF BLVD,27577,12
1,51772.00,9390.00,11561.38,064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,NC - Raleigh,SMITHFIELD,340090,JOHNSTON HEALTH,NC,509 BRIGHT LEAF BLVD,27577,13
2,29328.17,4803.29,7317.13,065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFA...,NC - Raleigh,SMITHFIELD,340090,JOHNSTON HEALTH,NC,509 BRIGHT LEAF BLVD,27577,24
3,28825.33,8597.08,10166.42,100 - SEIZURES W MCC,NC - Raleigh,SMITHFIELD,340090,JOHNSTON HEALTH,NC,509 BRIGHT LEAF BLVD,27577,12
4,82321.42,21691.17,23445.92,166 - OTHER RESP SYSTEM O.R. PROCEDURES W MCC,NC - Raleigh,SMITHFIELD,340090,JOHNSTON HEALTH,NC,509 BRIGHT LEAF BLVD,27577,12


In [53]:
nc_df.shape

(66, 12)

In [54]:
nc_df.dtypes

average_covered_charges                     object
average_medicare_payments                   object
average_total_payments                      object
drg_definition                              object
hospital_referral_region_hrr_description    object
provider_city                               object
provider_id                                 object
provider_name                               object
provider_state                              object
provider_street_address                     object
provider_zip_code                           object
total_discharges                            object
dtype: object

In [55]:
nc_df["average_medicare_payments"] = nc_df["average_medicare_payments"].astype("float")
nc_df["total_discharges"] = nc_df["total_discharges"].astype("int")

In [56]:
nc_df["medicare_total_paid"] = nc_df["average_medicare_payments"]*nc_df["total_discharges"]

In [57]:
# avg blended case rate for this sample
nc_df.medicare_total_paid.sum()/nc_df.total_discharges.sum()

8105.117789240972